In [5]:
import requests # requests機能をインポートします。
import pandas as pd
REQUEST_URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?bs=040&ar=030&ta=13&sc=13114&oz=13114013&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&page={}' #アクセス先をREQUEST_URLを代入します。
target_URL=REQUEST_URL.format(1)
res = requests.get(target_URL) #リクエストしたデータをresに代入します。
res.encoding = 'utf-8' # resの文字データがISO-8859-1なので、utf-8に変換して文字化けを防止します。
#ライブラリーインポート
from time import sleep
from bs4 import BeautifulSoup
import requests
import time

In [6]:
# アクセス先のURL
BASE_URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?bs=040&ar=030&ta=13&sc=13114&oz=13114013&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&page={}'

# 全ページの情報を格納するリスト
all_data = []

page_number = 1
d_list = []

while True:
    target_URL = BASE_URL.format(page_number)
    print('ページ', page_number, 'の情報を取得中...')

    r = requests.get(target_URL)
    if r.status_code != 200:
        print('ページ', page_number, 'の情報を取得できませんでした。終了します。')
        break

    soup = BeautifulSoup(r.text, 'html.parser')
    contents = soup.find_all('div', class_='cassetteitem')

    if not contents:
        print('ページ', page_number, 'の情報が見つかりませんでした。終了します。')
        break

    # ページごとの情報をリストに追加
    all_data.extend(contents)

    page_number += 1
    time.sleep(1)  # 相手のサーバーに負荷をかけないように1秒待機

    print('取得した情報の総数:', len(all_data))

    for content in contents:
        detail = content.find('div',class_='cassetteitem-detail') #物件・建物情報の変数に格納
        table = content.find('table',class_='cassetteitem_other') #各部屋の情報の変数に格納
        
        #物件・建物情報の抽出
        title = detail.find('div',class_='cassetteitem_content-title').text
        address = detail.find('li',class_='cassetteitem_detail-col1').text
        access = detail.find('li',class_='cassetteitem_detail-col2').text
        age = detail.find('li',class_='cassetteitem_detail-col3').text
        
        #各部屋の情報
        tr_tags = table.find_all('tr',class_='js-cassette_link')
    
        for tr_tag in tr_tags:
            floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
            fee, management_fee = price.find_all("li")
            deposit, gratuity = first_fee.find_all("li")
            madori, menseki = capacity.find_all("li")
            
            d = {
                'title':title,
                'address':address,
                'access':access,
                'age':age.strip(),
                'floor':floor.text.strip(),
                'fee':fee.text,
                'management_fee':management_fee.text,
                'deposit':deposit.text,
                'gratuity':gratuity.text,
                'madori':madori.text,
                'menseki':menseki.text
                }
            
            d_list.append(d)

ページ 1 の情報を取得中...
取得した情報の総数: 30
ページ 2 の情報を取得中...
取得した情報の総数: 60
ページ 3 の情報を取得中...
取得した情報の総数: 90
ページ 4 の情報を取得中...
取得した情報の総数: 120
ページ 5 の情報を取得中...
取得した情報の総数: 150
ページ 6 の情報を取得中...
取得した情報の総数: 180
ページ 7 の情報を取得中...
取得した情報の総数: 210
ページ 8 の情報を取得中...
取得した情報の総数: 240
ページ 9 の情報を取得中...
取得した情報の総数: 270
ページ 10 の情報を取得中...
取得した情報の総数: 300
ページ 11 の情報を取得中...
取得した情報の総数: 330
ページ 12 の情報を取得中...
取得した情報の総数: 360
ページ 13 の情報を取得中...
取得した情報の総数: 390
ページ 14 の情報を取得中...
取得した情報の総数: 420
ページ 15 の情報を取得中...
取得した情報の総数: 450
ページ 16 の情報を取得中...
取得した情報の総数: 480
ページ 17 の情報を取得中...
取得した情報の総数: 510
ページ 18 の情報を取得中...
取得した情報の総数: 540
ページ 19 の情報を取得中...
取得した情報の総数: 570
ページ 20 の情報を取得中...
取得した情報の総数: 600
ページ 21 の情報を取得中...
取得した情報の総数: 630
ページ 22 の情報を取得中...
取得した情報の総数: 660
ページ 23 の情報を取得中...
取得した情報の総数: 690
ページ 24 の情報を取得中...
取得した情報の総数: 720
ページ 25 の情報を取得中...
ページ 25 の情報が見つかりませんでした。終了します。


In [7]:
df = pd.DataFrame(d_list)

In [8]:
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,エクセル本町,東京都中野区本町２,\n東京メトロ丸ノ内線/中野坂上駅 歩4分\n都営大江戸線/西新宿五丁目駅 歩13分\n東京...,築71年\n2階建,1階,5.5万円,-,-,-,ワンルーム,15.7m2
1,生稲荘,東京都中野区本町２,\n東京メトロ丸ノ内線/中野坂上駅 歩7分\n都営大江戸線/西新宿五丁目駅 歩9分\n東京メ...,築64年\n2階建,2階,5.8万円,-,5.8万円,-,1DK,23.5m2
2,コーポ新中野,東京都中野区本町６,\n東京メトロ丸ノ内線/新中野駅 歩2分\nＪＲ総武線/中野駅 歩13分\n東京メトロ東西線...,築64年\n2階建,1階,5.2万円,2000円,-,-,ワンルーム,18.5m2
3,東京メトロ丸ノ内線 新中野駅 2階建 築59年,東京都中野区本町３,\n東京メトロ丸ノ内線/新中野駅 歩8分\n東京メトロ丸ノ内線/中野坂上駅 歩8分\n東京メ...,築59年\n2階建,1階,5.5万円,2000円,5.5万円,5.5万円,ワンルーム,23.2m2
4,ひまわり荘,東京都中野区本町３,\n東京メトロ丸ノ内線/新中野駅 歩8分\n東京メトロ丸ノ内線/中野坂上駅 歩8分\n東京メ...,築59年\n2階建,1階,5.5万円,2000円,5.5万円,5.5万円,ワンルーム,23.2m2


In [9]:
# CSV出力
df.to_csv('SUUMO_nakano.csv',index=None,encoding='utf-8-sig')


In [10]:
#Googleスプレットシートに出力
import gspread 
import json
from oauth2client.service_account import ServiceAccountCredentials

In [11]:
import time
SCOPE = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

#ダウンロードしたjsonファイルをドライブにアップデートした際のパス
CREDENTIAL_FILE = 'my-project-suumo-399411-35fa4a48a407.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIAL_FILE, SCOPE)

gc = gspread.authorize(credentials)

#書き込み先のスプレッドシートキーを追加
SPREADSHEET_KEY = '1IeKUZ1T0NgHlcmX5HwOPf0GH1VNKd4YlQXy1g8rb-rY'

#共有設定したスプレッドシートの1枚目のシートを開く
worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1

batch_size = 60  # 10個ずつ書き出すバッチサイズ
header_written = False  # ヘッダーが書き込まれたかどうかを示すフラグ
writeCount = 0  # ライトリクエストのカウンター

# データをGoogle Sheetsに書き込む
for i, d in enumerate(d_list, start=1):
    row_data = [d['title'], d['address'], d['access'], d['age'], d['floor'], d['fee'], d['management_fee'], d['deposit'], d['gratuity'], d['madori'], d['menseki']]
    
    # ヘッダーを書き込む
    if not header_written:
        header = ['タイトル', '住所', 'アクセス', '築年数', '階数', '家賃', '管理費', '敷金', '礼金', '間取り', '面積']
        worksheet.append_row(header)
        header_written = True
    
    worksheet.append_row(row_data)
    
    writeCount += 1  # ライトリクエストのカウントを増やす
    
    # ライトリクエストが一定数に達したら一時停止する
    if writeCount >= batch_size:
        print(f"{batch_size}個のデータを書き込みました。一時停止しています...")
        time.sleep(60)  # 60秒間一時停止
        writeCount = 0  # カウンターをリセット

print("情報がGoogleスプレッドシートに書き込まれました。")

60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
60個のデータを書き込みました。一時停止しています...
情報がGoogleスプレッドシートに書き込まれました。
